In [ ]:
import os, sys
sys.path.insert(0, "/home/isaiah/hana_mammograph/isaiah/")
from os.path import isdir, abspath, dirname
from collections import defaultdict
import numpy as np
import json
import csv
from addict import Dict
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ExponentialLR, CyclicLR
from torch.distributed import init_process_group, destroy_process_group
from torchlars import LARS
from dataset import MammoH5Data, GroupSampler
from models import DenseNet
from utils import printProgressBarRatio

import matplotlib.pyplot as plt

In [ ]:
datapath = "/home/isaiah/kaggle/mammo/preprocessed/mammodata224.h5"
metadatapath = "/home/isaiah/kaggle/mammo/preprocessed/metadata.json"
dataidspath = "/home/isaiah/kaggle/mammo/preprocessed/train_split.json"

metadata_params = Dict({
    "selected_columns": ['image_id', 'patient_id', 'laterality', 'view', 'age',
                       'cancer', 'implant', 'density', 'machine_id', 
                       'difficult_negative_case'],
    "default_value": 'na',
    "age_nan": "mean",
    "laterality_map": {'L': 0, 'R': 1},
    "view_map": {'CC': 0, 'MLO': 1},
    "density_map": {'A': 1, 'B': 2, 'C': 3, 'D': 4},
    "diff_neg_case_map": {"FALSE": 0, "TRUE": 1},
})

dataset_params = Dict({
    "augmentations": ["contrast_brightness"],
    "labels": ["cancer"],
    "sample_ratio": 0.5,
})
    
with open(dataidspath, "r") as f:
    data_ids = Dict(json.load(f))
    
classes = ['cancer', 'healthy']
batch_size = 4

In [ ]:
data = MammoH5Data("cpu", datapath, metadatapath,
                        dataset_params)
train_sampler = GroupSampler(data_ids["train"][classes[0]] + data_ids["train"][classes[1]], shuffle=True)
trainloader = DataLoader(data, batch_size, sampler=train_sampler)

In [ ]:
(img_id, img, gt) = next(iter(trainloader))
fig, axs = plt.subplots(2, 2, figsize=(14, 14))
axs[0, 0].set_title(str(img_id.detach().numpy()[0]))
axs[0, 0].imshow(img.detach().numpy()[0].squeeze(), cmap="bone")
axs[0, 1].set_title(str(img_id.detach().numpy()[1]))                    
axs[0, 1].imshow(img.detach().numpy()[1].squeeze(), cmap="bone")
axs[1, 0].set_title(str(img_id.detach().numpy()[2]))
axs[1, 0].imshow(img.detach().numpy()[2].squeeze(), cmap="bone")
axs[1, 1].set_title(str(img_id.detach().numpy()[3]))
axs[1, 1].imshow(img.detach().numpy()[3].squeeze(), cmap="bone")
plt.show()